In [33]:
import requests
import bs4 as bs
import pandas as pd
import numpy as np
import re
import json

In [51]:
data = pd.read_csv('data_all.csv', index_col=0)
data = data.rename(columns={'0': 'address', 'link': 'link', 'long': 'long', 'la':'lat'})

In [52]:
data.head()

,address,link,long,lat
0,汤臣一品,https://shanghai.anjuke.com/community/view/8,31.238608,121.508381
1,中粮海景壹号,https://shanghai.anjuke.com/community/view/218807,31.236882,121.510280
2,陆家嘴锦绣前城,https://shanghai.anjuke.com/community/view/725250,31.201611,121.542311
3,张江一村,https://shanghai.anjuke.com/community/view/13947,31.212369,121.619192
4,长岛花苑,https://shanghai.anjuke.com/community/view/2679,31.270386,121.609057


In [59]:
data['link'][4415]

'https://shanghai.anjuke.com/community/view/915073'

In [54]:
data ['property_type'] = pd.Series(np.empty(len(data)), index=data.index) 
data ['property_costs'] = pd.Series(np.empty(len(data)), index=data.index)
data ['total_area'] = pd.Series(np.empty(len(data)), index=data.index)
data ['number_of_units'] = pd.Series(np.empty(len(data)), index=data.index)
data ['year_built'] = pd.Series(np.empty(len(data)), index=data.index)
data ['parking_spots'] = pd.Series(np.empty(len(data)), index=data.index)
data ['volume_rate'] = pd.Series(np.empty(len(data)), index=data.index)
data ['green_rating'] = pd.Series(np.empty(len(data)), index=data.index)
data ['developer_company'] = pd.Series(np.empty(len(data)), index=data.index)
data ['property_management_company'] = pd.Series(np.empty(len(data)), index=data.index)

link = data['link'][0]
link

'https://shanghai.anjuke.com/community/view/8'

In [55]:
data.to_csv('data_populated_all_fixed.csv')

In [19]:
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:58.0) Gecko/20100101 Firefox/58.0',
    'From': 'data-x@gmail.com' 
}
source = requests.get(link, headers=headers).content

In [20]:
soup = bs.BeautifulSoup(source, 'html.parser')

In [21]:
parms = soup.find(class_="basic-parms-mod")

In [22]:
results = []
results = [i.contents[0] for i in parms.find_all("dd")]
results

['公寓',
 '15.00元/平米/月',
 '115000m²',
 '220户',
 '2006年',
 '458',
 '5.97',
 '40%(高)',
 '汤臣海景花园（上海浦东新区）有限公司',
 '晶华物业有限公司']

In [23]:
lookbehind = '(?<=new ajk.chart.priceTrend\(\{)'
getall = '[\s\S]+?'
lookahead = '(?=}\);)'
m = re.findall(lookbehind + getall + lookahead, str(soup))
print(m[1])


            icon       : ("https://pages.anjukestatic.com/usersite/site/img/global/2" || "") + "/char_circle.png",
            chartId    : $('#linechart'),
            houseItem  : $('.j-house-item'),
            commItem   : $('.j-community-item'),
            areaItem   : $('.j-area-item'),
            isCurrHouse: 0,
            avgDom     : $('.price-trend-mod .item .avg'),
            levelDom   : $('.price-trend-mod .item .price-stat'),
            percentDom : $('.price-trend-mod .item .percent'),
            arrowDom   : $('.price-trend-mod .item .status'),
            monthDom   : $('.price-trend-mod .item .month'),
            commId     : "8",
            commName   : "汤臣一品" || "",
            blockName  : "陆家嘴" || "",
            avgPrice   : "" || 0,
            nullDataId :["thishousenull","commhousenull", "areahousenull", "chartnoData"],
            descDataId :["thishousedesc","commhousedesc", "areahousedesc"],
            data : {"status":"ok","area":[{"201504":"3955

In [24]:
lookbehind = '(?<=data : {)'
getall = '[\s\S]+?'
lookahead = '(?=},\n)'
m2 = re.findall(lookbehind + getall + lookahead, str(m[1]))
print(m2[0])

"status":"ok","area":[{"201504":"39556"},{"201505":"41474"},{"201506":"42566"},{"201507":"42810"},{"201508":"43717"},{"201509":"44881"},{"201510":"45143"},{"201511":"46047"},{"201512":"48685"},{"201601":"52559"},{"201602":"54738"},{"201603":"64070"},{"201604":"63795"},{"201605":"63252"},{"201606":"63648"},{"201607":"65996"},{"201608":"67168"},{"201609":"69886"},{"201610":"69280"},{"201611":"70587"},{"201612":"71924"},{"201701":"73131"},{"201702":"70488"},{"201703":"71544"},{"201704":"73673"},{"201705":"77732"},{"201706":"79160"},{"201707":"77862"},{"201708":"74201"},{"201709":"73780"},{"201710":"73535"},{"201711":"73909"},{"201712":"72489"},{"201801":"71825"},{"201802":"73346"},{"201803":"73305"}],"community":[{"201504":"222603"},{"201505":"212010"},{"201506":"200504"},{"201507":"206136"},{"201508":"178042"},{"201509":"160504"},{"201510":"154106"},{"201511":"149121"},{"201512":"150532"},{"201601":"222540"},{"201602":"231768"},{"201603":"271283"},{"201604":"270121"},{"201605":"267822"},

In [25]:
price_history = json.loads('{' + m2[0] + '}')['community']
list(price_history[0].items())
data_list = [[0, data['0'][0], list(p.items())[0][0], list(p.items())[0][1]] for p in price_history]
data_list

[[0, '汤臣一品', '201504', '222603'],
 [0, '汤臣一品', '201505', '212010'],
 [0, '汤臣一品', '201506', '200504'],
 [0, '汤臣一品', '201507', '206136'],
 [0, '汤臣一品', '201508', '178042'],
 [0, '汤臣一品', '201509', '160504'],
 [0, '汤臣一品', '201510', '154106'],
 [0, '汤臣一品', '201511', '149121'],
 [0, '汤臣一品', '201512', '150532'],
 [0, '汤臣一品', '201601', '222540'],
 [0, '汤臣一品', '201602', '231768'],
 [0, '汤臣一品', '201603', '271283'],
 [0, '汤臣一品', '201604', '270121'],
 [0, '汤臣一品', '201605', '267822'],
 [0, '汤臣一品', '201606', '269502'],
 [0, '汤臣一品', '201607', '268787'],
 [0, '汤臣一品', '201608', '216301'],
 [0, '汤臣一品', '201609', '213342'],
 [0, '汤臣一品', '201610', '209459'],
 [0, '汤臣一品', '201611', '208361'],
 [0, '汤臣一品', '201612', '196184'],
 [0, '汤臣一品', '201701', '191220'],
 [0, '汤臣一品', '201702', '189084'],
 [0, '汤臣一品', '201703', '191410'],
 [0, '汤臣一品', '201704', '205951'],
 [0, '汤臣一品', '201705', '192041'],
 [0, '汤臣一品', '201706', '194957'],
 [0, '汤臣一品', '201707', '196681'],
 [0, '汤臣一品', '201708', '206819'],
 [0, '汤臣一品', '

In [26]:
labels = ['index', 'name', 'date', 'price']
df = pd.DataFrame.from_records(data_list, columns=labels)
df

,index,name,date,price
0,0,汤臣一品,201504,222603
1,0,汤臣一品,201505,212010
2,0,汤臣一品,201506,200504
3,0,汤臣一品,201507,206136
4,0,汤臣一品,201508,178042
5,0,汤臣一品,201509,160504
6,0,汤臣一品,201510,154106
7,0,汤臣一品,201511,149121
8,0,汤臣一品,201512,150532
9,0,汤臣一品,201601,222540


In [27]:
data.loc[0, 'Property type'] = results[0]
data.loc[0, 'Property costs'] = results[1]
data.loc[0, 'Total area'] = results[2]
data.loc[0, 'Number of units'] = results[3]
data.loc[0, 'Year built'] = results[4]
data.loc[0, 'Parking spots'] = results[5]
data.loc[0, 'Volume rate'] = results[6]
data.loc[0, 'Green rating'] = results[7]
data.loc[0, 'Developer Company'] = results[8]
data.loc[0, 'Property management company'] =results[9]
data.head()

,0,link,long,la,Property type,Property costs,Total area,Number of units,Year built,Parking spots,Volume rate,Green rating,Developer Company,Property management company
0,汤臣一品,https://shanghai.anjuke.com/community/view/8,31.238608,121.508381,公寓,15.00元/平米/月,115000m²,220户,2006年,458,5.97,40%(高),汤臣海景花园（上海浦东新区）有限公司,晶华物业有限公司
1,中粮海景壹号,https://shanghai.anjuke.com/community/view/218807,31.236882,121.510280,2.22649e-314,2.22649e-314,2.22649e-314,2.22649e-314,2.22649e-314,2.22649e-314,2.22649e-314,2.22649e-314,2.22649e-314,2.22649e-314
2,陆家嘴锦绣前城,https://shanghai.anjuke.com/community/view/725250,31.201611,121.542311,2.22649e-314,2.22649e-314,2.22649e-314,2.22649e-314,2.22649e-314,2.22649e-314,2.22649e-314,2.22649e-314,2.22649e-314,2.22649e-314
3,张江一村,https://shanghai.anjuke.com/community/view/13947,31.212369,121.619192,2.22649e-314,2.22649e-314,2.22649e-314,2.22649e-314,2.22649e-314,2.22649e-314,2.22649e-314,2.22649e-314,2.22649e-314,2.22649e-314
4,长岛花苑,https://shanghai.anjuke.com/community/view/2679,31.270386,121.609057,2.22649e-314,2.22649e-314,2.22649e-314,2.22649e-314,2.22649e-314,2.22649e-314,2.22649e-314,2.22649e-314,2.22649e-314,2.22649e-314


In [30]:
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:58.0) Gecko/20100101 Firefox/58.0',
    'From': 'data-x@gmail.com' 
}

price_labels = ['index', 'name', 'date', 'price']

def populate_row(row_index, data, soup):
    parms = soup.find(class_="basic-parms-mod")
    results = [i.contents[0] for i in parms.find_all("dd")]
    data.loc[row_index, 'Property type'] = results[0]
    data.loc[row_index, 'Property costs'] = results[1]
    data.loc[row_index, 'Total area'] = results[2]
    data.loc[row_index, 'Number of units'] = results[3]
    data.loc[row_index, 'Year built'] = results[4]
    data.loc[row_index, 'Parking spots'] = results[5]
    data.loc[row_index, 'Volume rate'] = results[6]
    data.loc[row_index, 'Green rating'] = results[7]
    data.loc[row_index, 'Developer Company'] = results[8]
    data.loc[row_index, 'Property management company'] =results[9]

def add_price(row_index, data, data_prices, soup):
    lookbehind = '(?<=new ajk.chart.priceTrend\(\{)'
    getall = '[\s\S]+?'
    lookahead = '(?=}\);)'
    m = re.findall(lookbehind + getall + lookahead, str(soup))
    lookbehind = '(?<=data : {)'
    getall = '[\s\S]+?'
    lookahead = '(?=},\n)'
    m2 = re.findall(lookbehind + getall + lookahead, str(m[1]))
    price_history = json.loads('{' + m2[0] + '}')['community']
    list(price_history[0].items())
    data_list = [[row_index, data['0'][row_index], list(p.items())[0][0], list(p.items())[0][1]] for p in price_history]
    df = pd.DataFrame.from_records(data_list, columns=labels)
    return data_prices.append(df, ignore_index=True)
    
def populate_batch_row(start = 0, batch_size = 1000):
    data = pd.read_csv('data_populated_all.csv', index_col=0)
    try:
        data_prices = pd.read_csv('data_prices.csv', index_col=0)
    except pd.errors.EmptyDataError:
        data_prices = pd.DataFrame()
    end = start + batch_size

    for i in range(start, end):
        try:
            link = data['link'][i]
            source = requests.get(link, headers=headers).content
            soup = bs.BeautifulSoup(source, 'html.parser')
            populate_row(i, data, soup)
            data_prices = add_price(i, data, data_prices, soup)
        except requests.ConnectionError:
            continue
        print("done with row ", i)
    data.to_csv('data_populated_all.csv')
    data_prices.to_csv('data_prices.csv')
    return (data, data_prices)

# change start to start at different batch start, batch size = 1000
data, data_prices = None, None

start, end, batch = 0, 20, 20
for i in range(start, end, batch):
    data, data_prices = populate_batch_row(i, batch)
# populate_batch_row(0, 1)

done with row  0
done with row  1
done with row  2
done with row  3
done with row  4
done with row  5
done with row  6
done with row  7
done with row  8
done with row  9
done with row  10
done with row  11
done with row  12
done with row  13
done with row  14
done with row  15
done with row  16
done with row  17
done with row  18
done with row  19
done with row  5
done with row  6
done with row  7
done with row  8
done with row  9
done with row  10
done with row  11
done with row  12
done with row  13
done with row  14
done with row  15
done with row  16
done with row  17
done with row  18
done with row  19


KeyboardInterrupt: 

In [31]:
data_prices

,index,name,date,price
0,0,汤臣一品,201504,222603
1,0,汤臣一品,201505,212010
2,0,汤臣一品,201506,200504
3,0,汤臣一品,201507,206136
4,0,汤臣一品,201508,178042
5,0,汤臣一品,201509,160504
6,0,汤臣一品,201510,154106
7,0,汤臣一品,201511,149121
8,0,汤臣一品,201512,150532
9,0,汤臣一品,201601,222540
